In [1]:
using JuMP
using CPLEX
using Distributions
using MathProgBase
#using GLPKMathProgInterface

## THE DIET PROBLEM (CLASSICAL LINEAR PROGRAMMING MODEL)

This model determines a least cost diet which meets the daily
allowances of nutrients for a moderately active man weighing 154 lbs.

We use the classical instance for the diet problem, proposed by:

Dantzig, G B, Chapter 27.1. In Linear Programming and Extensions. Princeton University Press, Princeton, New Jersey, 1963

Stigler's Nutrition model, the formulation is as follows:

$\min \sum_{j\in \mathbb{J}} c_j x_j$

s.t.

$\sum_{j \in \mathbb{J}} a_{ij}x_j \ge b_i, \quad \forall i \in \mathbb{I}$

$x_{j}\ge 0, \quad \forall ~j \in \mathbb{J}$

In [ ]:
A=[44.7  1411   2.0   365  0     55.4  33.3  441  0;
    36   897    1.7   99   30.9  17.4  7.9   106  0;
    8.4  422    15.1  9    26    3     23.5  11   60;
    20.6 17     0.6   6    55.8  0.2   0     0    0;
    7.4  448    16.4  19   28.1  0.8   10.3  4    0;
    15.7 661    1     48   0     9.6   8.1   471  0;
    41.7 0      0     0    0.2   0     0.5   5    0;
    2.2  333    0.2   139  169.2 6.4   50.8  316  525;
    4.4  249    0.3   37   0     18.2  3.6   79   0;
    5.8  705    6.8   45   3.5   1     4.9   209  0;
    2.4  138    3.7   80   69    4.3   5.8   37   862;
    2.6  125    4     36   7.2   9     4.5   26   5369;
    5.8  166    3.8   59   16.6  4.7   5.9   21   1184;
    14.3 336    1.8   118  6.7   29.4  7.1   198  2522;
    1.1  106    0     138  918.4 5.7   13.8  33   2755;
    9.6  138    2.7   54   290.7 8.4   5.4   83   1912;
    8.5  87     1.7   173  86.8  1.2   4.3   55   57;
    12.8 99     2.5   154  85.7  3.9   4.3   65   257;
    17.4 1055   3.7   459  5.1   26.9  38.2  93   0;
    26.9 1691   11.4  792  0     38.4  24.6  217  0]

A=A';

b=[30,70,0.8,12,5,1.8,2.7,18,75]

c=ones(20,1);

In [ ]:
A=[ 5   15;
    20   5;
    15   2]

b=[50,40,60]

c=[5,2];

Creating a range for the coefficient costs

In [2]:
function randomcoef(c)
    cl=zeros(length(c));
    cu=zeros(length(c));
    for i=1:length(c)
        cl[i]=c[i]-(c[i]*rand()/2);
        cu[i]=c[i]+(c[i]*rand()/2);
    end
    return cl,cu
end
#c_min,c_max=randomcoef(c)

randomcoef (generic function with 1 method)

From now on, we contruct our Min Max Regret model which is formulated as

$\min_{x} \max_{y,s} \sum_{j \in \mathbb{J}} c^s_j x_j - c^s_j y^s_j$

S.t.

$\sum_{j \in \mathbb{J}} a_{ij}x_j \ge b_i, \quad \forall i \in \mathbb{I}$

$\sum_{j \in \mathbb{J}} a_{ij}y^s_j \ge b_i, \quad \forall i \in \mathbb{i}, \forall s \in \mathbb{S}$

$x_{j}\ge 0, \quad \forall ~j \in \mathbb{J}$

$y^s_{j}\ge 0, \quad \forall ~j \in \mathbb{J}, \forall s \in \mathbb{S}$

$\underline{c}_j\le c^s_j \le \overline{c}_j, \quad \forall ~j \in \mathbb{J}, \forall s \in \mathbb{S}$

In [6]:
function solveLP(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

#Fixing some x
function solveLP(A,b,c,solver,v,max)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    @constraint(submodel, sum(v[j]*x[j] for j in 1:n)==max)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor,sol=getobjectivevalue(submodel),getvalue(x)
    return valor,sol
end

solveLP (generic function with 2 methods)

We'll calculate the upper bounds for $x_j$, for this purpose, a LP is solved fixing $x_j$ at its empirical maximum

$\hat{x}_j=\max_{i:a_{ij}>0}\Big\{\frac{b_j}{a_{ij}}\Big\},\qquad \forall ~j \in J$

In [7]:
xmax=zeros(length(c));
for j=1:length(c)
    v=zeros(length(c));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    pos=find(x->x==xm,b./A[:,j])
    pos1=find(x->x==xm,maxim)
    new=xm-Ar[pos]
    if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
        xmax[j]=new[1]
    else
        xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    end
end
println(xmax)

[39.935,42.9525,45.5621,41.4995,27.309,29.859,28.258,54.5871,45.0542,27.5793,47.1041,37.9494,39.071,25.9467,37.7268,42.9525,31.2754,30.266,44.0277,47.1041,46.723,32.4863,34.4977,31.6582,42.2995,30.3675,27.8551,36.3531,64.3517,22.1295,43.555,37.539,58.4415,57.4962,30.6406,36.8953,40.028,40.028,32.7433,50.383,32.2196,60.735,45.7334,48.2968,46.024,38.9908,31.9017,36.0755,45.0542,31.3195,42.9525,50.383,51.6327,28.472,34.05,35.2318,80.4947,29.4741,57.53,46.3143,39.071,42.7759,49.9118,57.4962,41.5826,29.6248,45.4892,44.1601,40.5488,34.496,27.3885,30.0654,30.308,42.9525,44.7086,54.4492,78.142,53.9028,36.376,40.5245,46.024,39.719,37.7319,57.4962,36.1278,40.2745,36.236,42.7759,32.8884,39.663,55.4806,45.0542,57.53,93.878,47.7819,42.7815,64.3517,54.3508,62.3536,42.7815,36.5885,59.37,34.05,37.539,29.388,44.2926,29.388,35.4372,24.726,35.6155,25.5893,30.308,39.071,30.6506,35.5269,54.5871,61.7183,43.555,49.6406,39.663,37.539,27.0566,50.383,43.3151,64.3517,47.523,49.5868,39.663,38.1264,32.1758,31.8935

In [8]:
#Subproblem (NP-hard) function 
function SubproblemLP(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b)
    r=size(A,1)
    tol=1e-6
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)
    
    ind=[]
    for j=1:n
        if fix1[j]>=1-tol
            #println("upper")
            setvalue(zup[j],1)
            setvalue(zun[j],0)
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]>=1-tol
                #println("under")
                setvalue(zup[j],0)
                setvalue(zun[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            else
                push!(ind,j)
                #println("Algo pasa")
            end
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], (xmax[j]-xbar[j])*zup[j] >= yup[j])
    @constraint(m, minusy[j in ind], xbar[j]*zun[j] >= yun[j])
    
    #print("-----------------SUB-------------------")
    #println(m)
    #print("-----------------SUB-------------------")
    
    status = solve(m)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    #println("valor z ",zu)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end    
    return valor,cost_scen
end

SubproblemLP (generic function with 1 method)

In [9]:
function solveRLP(A,b,cminus, cplus, xmax, n::Int64, solver=CplexSolver(CPX_PARAM_LPMETHOD=1))
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveLP(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    @constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)        
    #print(m)
    
    status = solve(m)
    #println(status)
    #println(getvalue(x))
    println("LB: ", getobjectivevalue(m))
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB<=tol
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    println("Prueba LB",abs(LB-LBprime))
    while abs(LB-LBprime)>=tol#conteo<=10
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]>=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\t",fixup,"\n")
        #println("\t\t",fixun,"\n")
        tt=time()
        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x_val,b)
        println("Tiempo subproblema: ", time()-tt)
        println("Costo optimo subproblema: ", optimalCost)
        #New cut
        tt1=time()
        rhs=solveLP(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        println("Tiempo RHS: ", time()-tt1)
        @constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs )
        
        #print("-----------------######-------------------")
        #print(m)
        #print("-----------------######-------------------")
        
        tt2=time()
        status = solve(m)
        println("Tiempo LP: ", time()-tt2)
        println(status)
        #println(getvalue(x))
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        #if abs(LB-LBprime)<=tol
        #    conteo += 1
        #else
        #    conteo = 1
        #end
        println("LB: ", getobjectivevalue(m))
        it += 1
        #print("GAP entre soluciones consecutivas", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

solveRLP (generic function with 2 methods)

In [10]:
solveRLP(A,b,c_min, c_max, xmax, length(c))

CPXPARAM_LPMethod                                1
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 1 columns.
Reduced LP has 480 rows, 600 columns, and 278437 nonzeros.
Presolve time = 0.22 sec. (31.30 ticks)

Iteration log . . .
Iteration:     1    Scaled infeas =            14.835535
Iteration:     8    Objective     =          1849.610917
LB: -3.7306690738754696e-15
Prueba LB0.9999999999999967
Tiempo subproblema: 11.499998950958252
Costo optimo subproblema: 4.946175153698256
Tiempo RHS: 0.5731499195098877
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =             0.000000
Iteration:     2    Objective     =             4.942340
Tiempo LP: 1.048902988433838
Optimal
LB: 0.6338729601075019
Tiempo subproblema: 6.68617395401001
Costo optimo subproblema: 7.032151182108639
Tiempo RHS: 0.1826299209594727
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_LPMethod    

LoadError: InterruptException:

In [ ]:
#Modelo completo
A=[ 5   15;
    20   5;
    15   2]
b=[50,40,60]
c=[7.5,2];
c_min=[6,1]
c_max=[9,3]

y1=solveLP(A,b,[6,1],CplexSolver(CPX_PARAM_SCRIND=0))
println("y1 opt: ", y1)
y2=solveLP(A,b,[6,3],CplexSolver(CPX_PARAM_SCRIND=0))
println("y2 opt: ", y2)
y3=solveLP(A,b,[9,1],CplexSolver(CPX_PARAM_SCRIND=0))
println("y3 opt: ", y3)
y4=solveLP(A,b,[9,3],CplexSolver(CPX_PARAM_SCRIND=0))
println("y4 opt: ", y4)

M=Model(solver=CplexSolver(CPX_PARAM_SCRIND=0))

@variable(M, x[1:2]>=0)
@variable(M, t)

@objective(M, Min, t)

@constraint(M, req[i = 1:3], sum(A[i,j]*x[j] for j in 1:2) >= b[i])
@constraint(M, t >= 6*x[1]+x[2]-y1)
@constraint(M, t >= 6*x[1]+3*x[2]-y2)
@constraint(M, t >= 9*x[1]+x[2]-y3)
@constraint(M, t >= 9*x[1]+3*x[2]-y4)

print(M)

status = solve(M)
println(getobjectivevalue(M))
println(getvalue(x))
println(A*getvalue(x))

In [11]:
function check_solution(zval,cun,cup)
    cost_scen=zeros(Float64,length(zval))
    tol=1e-6
    intsol=true
    for i=1:length(zval)
        if zval[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            if zval[i]<=tol
                cost_scen[i]=cup[i]
            else
                intsol=false
            end
        end
    end
    return intsol,cost_scen
end

function check_distance(scens,costscen,mindis)
    setdis=true
    tol=1e-6
    for i=1:size(scens,1)
        cont=0
        for j=1:size(scens,2)
            if costscen[j]>=scens[i,j]-tol && costscen[j]<=scens[i,j]+tol
                cont +=1
                println(cont)
            end
        end
        if cont/size(scens,2)>mindis
            setdis=false
            return setdis
        end
    end
    return setdis
end

check_distance (generic function with 1 method)

In [18]:
type NodeData
    time::Float64  # in seconds since the epoch
    obj::Float64
end

#Subproblem (NP-hard) function 
function SubproblemLPim(cup,cun,solver,fix1,fix2,n::Int64,xmax,xbar,b,zopt,tt,scens,timemax,mindis)
    r=size(A,1)
    tol=1e-6
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)
    
    ind=[]
    for j=1:n
        if fix1[j]>=1-tol
            setvalue(zup[j],1)
            setvalue(zun[j],0)
            setupperbound(yup[j],xmax[j]-xbar[j])
            setupperbound(yun[j],0)
        else
            if fix2[j]>=1-tol
                #println("under")
                setvalue(zup[j],0)
                setvalue(zun[j],1)
                setupperbound(yup[j],0)
                setupperbound(yun[j],xbar[j])
            else
                push!(ind,j)
                #println("Algo pasa")
            end
        end
    end
    
    @objective(m,Max,sum(cup[j]*yun[j] for j in 1:n) - sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*xbar[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yun[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j in ind], zup[j]+zun[j]==1)
    @constraint(m, plusy[j in ind], (xmax[j]-xbar[j])*zup[j] >= yup[j])
    @constraint(m, minusy[j in ind], xbar[j]*zun[j] >= yun[j])
    
    #print("-----------------SUB-------------------")
    #println(m)
    #print("-----------------SUB-------------------")
    
    #function getcursol(cb)
    #    zval=getvalue(zup)
    #    obj=MathProgBase.cbgetobj(cb)
    #    cost_scen=zeros(length(zval))
    #    println("Solución parcial: ", obj)
    #    println("zval ", zval)
    #    
    #    intsol, cost_scen = check_solution(zval,cun,cup)
    #    println("Es entera la solución:", intsol)
    #    
    #    setdis = check_distance(scens,cost_scen,mindis)
    #    println("Cumple condiciones de distancia: ", setdis)
    #    
    #    if (obj-zopt)/zopt>0.1 && intsol==true && setdis==true
    #        println("Solucion satisface condiciones para agregar el escenario")
    #        return JuMP.StopTheSolver
    #    end
    #end
    
    #if time()-tt<=timemax
    #    addlazycallback(m,getcursol)
    #end
    bbdata = NodeData[]
    
    
    function infocallback(cb)
        obj       = MathProgBase.cbgetobj(cb)
        push!(bbdata, NodeData(time(),obj))
    end
    addinfocallback(m, infocallback, when = :MIPNode)
    
    println(bbdata)

    status = solve(m)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    #println("valor z ",zu)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]>=1-tol
            cost_scen[i]=cun[i]
        else
            cost_scen[i]=cup[i]
        end
    end     
    return valor,cost_scen
end

SubproblemLPim (generic function with 1 method)

In [19]:
function solveRLPim(A,b,cminus, cplus, xmax, n::Int64,timemax,mindis,solver=CplexSolver(CPX_PARAM_LPMETHOD=1))
    #We initilize the algorithm solving the original LP using the midpoint scenario
    r=size(A,1)
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveLP(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    
    #We set the MinMax Regret problem for the first iteration
    m = Model(solver = solver)
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    @constraint(m,θ - sum(cMidPoint[j]*x[j] for j in 1:n) >=rhs)        
    #print(m)
    
    status = solve(m)
    #println(status)
    #println(getvalue(x))
    println("LB: ", getobjectivevalue(m))
    
    #Max admisible difference between consecutive solutions of our MinMax Regret Problem
    tol=0.0001
    
    #Initial bounds
    LB=getobjectivevalue(m)
    if LB<=tol
        LBprime=-1
    else
        LBprime=-10*LB
    end
    
    #Tolerance to 0.0
    EPSILON=0.0001
    it=1
    
    conteo=0
    
    println("Prueba LB",abs(LB-LBprime))
    while abs(LB-LBprime)>=tol#conteo<=10
        #Generating a new cut
        rhs=0
        x_val = getvalue(x)
        #Analizing x we can fix some value of z+ and z- in the combinatorial subproblem
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<=EPSILON 
                fixun[i]=1
            else
                if x_val[i]>=(xmax[i]-EPSILON)
                    fixup[i]=1
                end
            end
        end
        #println("\t\t",fixup,"\n")
        #println("\t\t",fixun,"\n")
        if it==1
            tta=time()
            optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x_val,b)
            println("Tiempo subproblema: ", time()-tta)
            scens=cost
        else
            tta=time()
            optimalcost,cost = SubproblemLPim(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x_val,b,getobjectivevalue(m),time(),scens,timemax,mindis)
            println("Tiempo subproblema: ", time()-tta)
            scens=vcat(scens,cost)
        end
        println("Costo optimo subproblema: ", optimalCost)
        #New cut
        tt1=time()
        rhs=solveLP(A,b,cost,CplexSolver(CPX_PARAM_SCRIND=0))
        println("Tiempo RHS: ", time()-tt1)
        @constraint(m, θ - sum(cost[j]*x[j] for j in 1:n) >= -rhs )
        
        #print("-----------------######-------------------")
        #print(m)
        #print("-----------------######-------------------")
        
        tt2=time()
        status = solve(m)
        println("Tiempo LP: ", time()-tt2)
        println(status)
        #println(getvalue(x))
        #Updating bounds
        LBprime=LB
        LB=getobjectivevalue(m)
        
        #Counting consecutive LPs with small improvements (tol) in our bounds
        #if abs(LB-LBprime)<=tol
        #    conteo += 1
        #else
        #    conteo = 1
        #end
        println("LB: ", getobjectivevalue(m))
        it += 1
        #print("GAP entre soluciones consecutivas", abs(LB-LBprime))
    end
    
    println("TotalIteraciones: ", it)
    println("maxRegret: ",getobjectivevalue(m))
    println("Solución",getvalue(x))
end

solveRLPim (generic function with 2 methods)

In [ ]:
solveRLPim(A,b,c_min, c_max, xmax, length(c),10,0.1)

CPXPARAM_LPMethod                                1
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 1 columns.
Reduced LP has 480 rows, 600 columns, and 278437 nonzeros.
Presolve time = 0.23 sec. (31.30 ticks)

Iteration log . . .
Iteration:     1    Scaled infeas =            14.835535
Iteration:     8    Objective     =          1849.610917
LB: -3.3306690738754696e-15
Prueba LB0.9999999999999967
Tiempo subproblema: 7.666306953430176
Costo optimo subproblema: 4.946175153698256
Tiempo RHS: 0.626917839050293
CPLEX Error  3003: Not a mixed-integer problem.
CPXPARAM_LPMethod                                1

Iteration log . . .
Iteration:     1    Scaled infeas =             0.000000
Iteration:     2    Objective     =             4.942340
Tiempo LP: 0.1183919906616211
Optimal
LB: 0.6338729601075019
NodeData[]
CPLEX Error  3003: Not a mixed-integer problem.
Tiempo subproblema: 9.651810960769653
Costo optimo subproblema: 4.946175153698256
Tiempo RHS: 0.6562139987945557
CPXPARAM_L

In [3]:
function buildinstance(n,m)
    #Vector de costos de primera etapa
    c=zeros(n)
    for i=1:n
        min=round(rand()*10)
        max=min+1+round(rand()*min*10)
        c[i]=round(rand(Uniform(min,max)))
        if c[i]==0
            c[i]=1
        end
    end

    #A matrix
    A=round(rand(1,n)*rand(Uniform(1,30)))
    for i=2:m
        min=round(rand()*rand(Uniform(1,30)))
        max=min+1+round(rand()*min*rand(Uniform(1,30)))
        A=vcat(A,round(rand(Uniform(min,max),1,n)))
    end

    #Set b vector
    incap=sum(A,2)
    b=round((incap.*round(rand(Uniform(5,20),m,1))).*rand(m))
    return c,A,b
end

buildinstance (generic function with 1 method)

In [4]:
c,A,b=buildinstance(600,480)
c_min,c_max=randomcoef(c)

([15.3087,56.3713,3.36003,30.8391,9.44646,1.62924,17.7572,18.7035,16.9316,18.8202  …  23.6594,5.16577,31.1451,4.69704,12.3786,6.69545,13.9363,6.15512,4.23125,2.79593],[31.7777,70.6141,8.13105,49.3215,15.0613,2.52607,21.1685,23.7899,23.8972,30.8722  …  39.105,6.96405,38.6027,8.60773,19.6984,17.9674,23.535,11.8286,9.51433,5.49002])

In [ ]:
c

In [ ]:
c_max

In [ ]:
c_min

In [5]:
A=A
b=b/1000

480×1 Array{Float64,2}:
   2.162
 151.441
   0.615
 120.049
   2.128
   8.436
 112.421
  23.005
  80.578
 409.973
  29.629
   6.739
  93.144
   ⋮    
  72.752
  67.107
 197.742
   5.012
 569.848
 706.561
 825.478
  79.188
  55.632
  58.421
 545.871
 140.131